# **Find My Pic - ABHILASH SPOLE**
*Using ChromaDB and Vector Emebddings*

In [ ]:
%%capture
!pip install chromadb
!pip install pillow
!pip install open-clip-torch
!pip install matplotlib
!pip install gradio

In [ ]:
import os
import shutil
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from matplotlib import pyplot as plt
import gradio as gr
from PIL import Image

images_folder = '/content/drive/MyDrive/fotos_folder'
img_files = os.listdir(images_folder)

In [ ]:
# Create database file at folder "my_vectordb"
chroma_client_cnm = chromadb.PersistentClient(path="my_vector_db")
chroma_client_cnm.read_only = False

# Instantiate image loader helper.
image_loader = ImageLoader()
# Instantiate multimodal embedding function.
multimodal_fnc = OpenCLIPEmbeddingFunction()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
# Create the imeags database to store the embeddings
images_db = chroma_client_cnm.get_or_create_collection(name="images_db", embedding_function=multimodal_fnc, data_loader=image_loader)

In [ ]:
# Add the images to the database
for file in img_files:
    file_path = os.path.join(images_folder, file)
    images_db.add(
        ids=[file],
        uris=[file_path],
        metadatas=[{'img_category': 'everything'}]

    )

In [ ]:
print("The Number of Images in your folder is", images_db.count())

The Number of Images in your folder is 13


In [ ]:
def search_images(query):
    query_results = images_db.query(
        query_texts=[query],
        n_results=2,
        include=['documents', 'distances', 'metadatas', 'data', 'uris']
    )

    result_uris = query_results['uris'][0]
    result_images = [Image.open(uri) for uri in result_uris]

    return result_images

iface = gr.Interface(
    fn=search_images,
    inputs=gr.Textbox(label="Search Query"),
    outputs=gr.Gallery(label="Matching Images"),
    title="Local Image Search Tool",
    description="Enter a description of the image you are looking for."
)

iface.launch(debug=True)
